In [47]:
!pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/e3/a9/3bfb8f52fee7ab0950bb94920776dbca00b9515ff5fa504374c059d6192c/lightgbm-4.3.0-py3-none-macosx_10_15_x86_64.macosx_11_6_x86_64.macosx_12_5_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.8 MB/s eta 0:00:00a 0:00:01


In [48]:
import lightgbm as lgb

OSError: dlopen(/usr/local/anaconda3/lib/python3.11/site-packages/lightgbm/lib/lib_lightgbm.so, 0x0006): Library not loaded: '/usr/local/opt/libomp/lib/libomp.dylib'
  Referenced from: '/usr/local/anaconda3/lib/python3.11/site-packages/lightgbm/lib/lib_lightgbm.so'
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

In [1]:
import numpy as np
import pandas as pd
import os 
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import tensorflow as tf
from keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing import image
from keras.applications.resnet import preprocess_input, decode_predictions

2024-04-03 20:39:18.228368: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
print(f"numpy-{np.__version__}")
print(f"pandas-{pd.__version__}")
print(f"keras-{keras.__version__}")
print(f"tensorflow-{tf.__version__}")

numpy-1.24.3
pandas-1.5.3
keras-3.1.1
tensorflow-2.16.1


The ResNet50 model without the top layers (specified by include_top=False) acts as a feature extractor. It takes preprocessed images as input and produces feature maps as output, where each feature map represents higher-level features learned by the model. These feature maps capture various aspects of the input images, such as edges, textures, and object parts.

In [17]:
model = ResNet50(weights='imagenet', include_top=False)

In [5]:
# testing on one image
img_path = 'test_images/110369.jpg'
img = image.load_img(img_path, target_size=(224, 224))
image_array = image.img_to_array(img) # image to np array
images_array = np.expand_dims(image_array, axis=0) # to add batch size [img1, img2, img3, ... img100]
images = np.array([image_array, image_array, image_array, image_array, image_array])
preprocessed_images_array = preprocess_input(images) # ResNet50 pre-processing: normalisation, BGR format
embedding = model.predict(preprocessed_images_array) # features map / embedding
embedding_flat = embedding.flatten()

1/1 [==============================] - 5s 5s/step


In [ ]:
def strip_and_convert(filename):
    return int(filename.split('.')[0])

In [53]:
# load images
directory = 'images'
image_name_list = os.listdir(directory)
image_name_list.sort(key=strip_and_convert)

In [56]:
# generate partitions
n = 1024
l = len(image_name_list)
partitions = np.array_split(image_name_list, range(n, l, n))

In [60]:
# process partitions 
partition_number = 0
for partition in partitions:
    print(f"processing partition {partition_number} ({len(partition)} images)...")
    
    embedding_list = []
    images = []
    for image_name in partition:
        img_path = os.path.join(directory, image_name)
        img = image.load_img(img_path, target_size=(224, 224))
        image_array = image.img_to_array(img)
        images.append(image_array)
    
    images_array = np.array(images)
    preprocessed_images_array = preprocess_input(images_array) # ResNet50 pre-processing: normalisation, BGR format
    embeddings = model.predict(preprocessed_images_array) # features map / embedding
    
    for embedding in embeddings:
        embedding_flat = embedding.flatten()
        embedding_list.append(embedding_flat)

    df = pd.DataFrame({
        'ImageName': partition,
        'Embedding': embedding_list})
    df['ImageName'] = df['ImageName'].apply(strip_and_convert)
    

    print(f"saving results for partition {partition_number}...")
    filename = f"./embeddings_{partition_number}.pickle.gz"
    df.to_pickle(filename)
    print(f"finished processing partition {partition_number}")
    partition_number += 1
    

processing partition 0 (32 images)...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
saving results for partition 0...
finished processing partition 0
processing partition 1 (32 images)...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
saving results for partition 1...
finished processing partition 1


In [62]:
# check we can read the pickle file
df_2 = pd.read_pickle("./embeddings_0.pickle.gz")

In [63]:
df_2.head()

,ImageName,Embedding
0,0,"[0.0, 2.6630778, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [65]:
len(image_name_list)/1024

156.5712890625

In [2]:
# creating slice of images based on cleaning from 02/04/24
cleaned_df = pd.read_csv('English_fiction_pre_PCA_3.csv')

In [4]:
cleaned_df.shape

(26774, 18)

In [10]:
cleaned_df.head()

,Title,description,authors,image,previewLink,publisher,infoLink,categories,index,reviews number,average rating,median rating,min review date,max review date,weighted rating,date,year,description_language
0,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['fiction'],3,32,3.718750,5.0,2005-02-14,2006-07-01,3.938400,2005-02-01,2005.0,English
1,The Forbidden Stories of Marta Veneranda,"Marta Veneranda, a Latina neoyorkina, finds th...",['Sonia Rivera-Valdes'],http://books.google.com/books/content?id=A7aYb...,http://books.google.nl/books?id=A7aYbAvagu8C&p...,Seven Stories Press,http://books.google.nl/books?id=A7aYbAvagu8C&d...,['fiction'],24,1,5.000000,5.0,2005-01-24,2005-01-24,4.306145,2001-03-06,2001.0,English
2,Tess and the Highlander,"In 1543, on a windswept isle off of Scotland, ...",['May Mcgoldrick'],http://books.google.com/books/content?id=VmCRS...,http://books.google.nl/books?id=VmCRSPmY3WkC&d...,Harper Collins,http://books.google.nl/books?id=VmCRSPmY3WkC&d...,['juvenile fiction'],42,17,4.235294,5.0,2002-10-22,2011-05-25,4.256189,2002-11-01,2002.0,English
3,"Eight Men And A Lady (Elizabeth Sinclair, Harl...",Eight Men And A Lady by Elizabeth Sinclair rel...,['Elizabeth Sinclair'],http://books.google.com/books/content?id=Z6uzJ...,http://books.google.nl/books?id=Z6uzJgLWViUC&q...,Harlequin Treasury-Harlequin American Romance 90s,http://books.google.nl/books?id=Z6uzJgLWViUC&d...,['fiction'],49,2,5.000000,5.0,1998-04-16,2000-05-14,4.336313,1997-01-01,1997.0,English
4,Night World: Daughters Of Darkness,"""There’s something strange about the new girls...",['L.J. Smith'],http://books.google.com/books/content?id=c9icD...,http://books.google.nl/books?id=c9icDQAAQBAJ&p...,Simon and Schuster,http://books.google.nl/books?id=c9icDQAAQBAJ&d...,['juvenile fiction'],73,134,4.768657,5.0,1996-08-07,2012-09-18,4.701517,2016-12-06,2016.0,English


In [41]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26774 entries, 0 to 26773
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Title                 26774 non-null  object 
 1   description           26774 non-null  object 
 2   authors               26774 non-null  object 
 3   image                 26774 non-null  object 
 4   previewLink           26774 non-null  object 
 5   publisher             26774 non-null  object 
 6   infoLink              26774 non-null  object 
 7   categories            26774 non-null  object 
 8   index                 26774 non-null  int64  
 9   reviews number        26774 non-null  int64  
 10  average rating        26774 non-null  float64
 11  median rating         26774 non-null  float64
 12  min review date       26774 non-null  object 
 13  max review date       26774 non-null  object 
 14  weighted rating       26774 non-null  float64
 15  date               

In [21]:
indices_cleaned_df = cleaned_df['index'].to_list()

In [23]:
indices_cleaned_df[:10]

[3, 24, 42, 49, 73, 83, 115, 116, 131, 132]

In [24]:
len(indices_cleaned_df)

26774

In [28]:
directory = 'results'
results_files = os.listdir(directory)

In [30]:
len(results_files)

157

In [34]:
filtered_embeddings = []

# read one pickle batch
for result_file in results_files:
    result_path = os.path.join(directory, result_file)
    df = pd.read_pickle(result_path)
    filtered_df = df[df['ImageName'].isin(indices_cleaned_df)]
    filtered_embeddings.append(filtered_df)


In [35]:
len(filtered_embeddings)

157

In [36]:
filtered_embeddings_df = pd.concat(filtered_embeddings, ignore_index=True)

In [37]:
filtered_embeddings_df.shape

(26774, 2)

In [38]:
filtered_embeddings_df.head()

,ImageName,Embedding
0,134733,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,134736,"[0.65182924, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,134739,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,134741,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,134743,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5643275, 0.0,..."


In [39]:
filtered_embeddings_df.sort_values('ImageName', inplace=True)

In [40]:
filtered_embeddings_df.head()

,ImageName,Embedding
24609,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24610,24,"[1.1416551, 0.0, 0.0, 0.0, 0.0, 0.0, 2.57376, ..."
24611,42,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.2108593, 0.0,..."
24612,49,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24613,73,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [49]:
filtered_embeddings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26774 entries, 24609 to 9626
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ImageName  26774 non-null  int64 
 1   Embedding  26774 non-null  object
dtypes: int64(1), object(1)
memory usage: 627.5+ KB


In [43]:
cleaned_df_with_embeddings = cleaned_df.set_index('index').join(filtered_embeddings_df.set_index('ImageName'))

In [45]:
cleaned_df_with_embeddings.shape

(26774, 18)

In [50]:
cleaned_df_with_embeddings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26774 entries, 3 to 212402
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Title                 26774 non-null  object 
 1   description           26774 non-null  object 
 2   authors               26774 non-null  object 
 3   image                 26774 non-null  object 
 4   previewLink           26774 non-null  object 
 5   publisher             26774 non-null  object 
 6   infoLink              26774 non-null  object 
 7   categories            26774 non-null  object 
 8   reviews number        26774 non-null  int64  
 9   average rating        26774 non-null  float64
 10  median rating         26774 non-null  float64
 11  min review date       26774 non-null  object 
 12  max review date       26774 non-null  object 
 13  weighted rating       26774 non-null  float64
 14  date                  26774 non-null  object 
 15  year              

In [46]:
cleaned_df_with_embeddings.to_pickle('English_fiction_pre_PCA_3_with_embeddings.gz')

In [51]:
df_embedding_batch_0 = pd.read_pickle('results/embeddings_0.pickle.gz')

In [52]:
df_embedding_batch_0.head()

,ImageName,Embedding
0,0,"[0.0, 2.6699476, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [53]:
df_embedding_batch_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ImageName  1024 non-null   int64 
 1   Embedding  1024 non-null   object
dtypes: int64(1), object(1)
memory usage: 16.1+ KB


In [54]:
len(df_embedding_batch_0['Embedding'][0])

100352

In [55]:
type(df_embedding_batch_0['Embedding'][0])

numpy.ndarray